In [5]:
import pandas as pd
import numpy as np
import sys
import math
import csv

wang_data = pd.read_csv("../../wang.txt", comment="#", delim_whitespace=True, header=None, skiprows=1)
cos_ang = wang_data.iloc[:,0]
kang_rad = wang_data.iloc[:,1]
weight = wang_data.iloc[:,2]

def asymmetry(fname):
    data = pd.read_csv(fname, comment="#", delim_whitespace=True, header=None, skiprows=3)


    with open(fname, "r") as fp:
        fp.readline()
        num_krad = int(fp.readline().strip("\n"))
        num_kang = int(fp.readline().strip("\n"))

    krad = data.iloc[:,0]
    kang = data.iloc[:,1]
    kval = data.iloc[:,2]

    krad = krad.values.reshape([num_krad,num_kang])
    kang = kang.values.reshape([num_krad, num_kang])
    kval = kval.values.reshape([num_krad, num_kang])

    num_krad_half = int(num_krad/2)
    num_kang_half = int(num_kang/2)

    momentum = krad[:,0]
    orbital_2s = 0
    dk = np.zeros((num_krad), dtype = 'float')
    weighted_kval = np.zeros((num_krad, num_kang), dtype = 'float')
    pie1 = pie2 = pie3 = pie4 = 0

    for i in range(0,num_krad):
        if(i < num_krad):
            dk[i] = momentum[i] - momentum[i-1]   
        if(i == 0) :
            dk[i] = momentum[i] - 0.0

    for i in range(0, num_krad):
        for j in range(0, num_kang):
            weighted_kval[i, j]= kval[i, j] * weight[j]


    for i in range(0, num_krad_half):
        for j in range(0, num_kang_half):
            pie1 +=  weighted_kval[i, j] * momentum[i] * momentum[i] * dk[i]
        for j in range(num_kang_half, num_kang):
            pie2 +=  weighted_kval[i, j] * momentum[i] * momentum[i] * dk[i]

    for i in range(num_kang_half, num_krad):
        for j in range(0, num_kang_half):
            pie3 +=  weighted_kval[i, j] * momentum[i] * momentum[i] * dk[i]
        for j in range(num_kang_half, num_kang):
            pie4 +=  weighted_kval[i, j] * momentum[i] * momentum[i] * dk[i]

    lr_asymmetry1 = (pie2 - pie1)/(pie1 + pie2)

    return lr_asymmetry1

name = ['cep90','cep45','cep0','cep_45','cep_90','cep_135','cep_180','cep_225']
asym = []
for element in name:
    fname = '/Ne.1.0.8.SPECTRUM/Ne.1.0.8_00000030.ang_spec'
    fname = element + fname 
    asym.append(asymmetry(fname))   

In [6]:
from scipy.special import *
def Beta(fname):
    data = pd.read_csv(fname, comment="#", delim_whitespace=True, header=None, skiprows=3)
    with open(fname, "r") as fp:
        fp.readline()
        num_krad = int(fp.readline().strip("\n"))
        num_kang = int(fp.readline().strip("\n"))

    krad = data.iloc[:,0]
    kang = data.iloc[:,1]
    kval = data.iloc[:,2]
    
    krad = krad.values.reshape([num_krad,num_kang])
    kang = kang.values.reshape([num_krad, num_kang])
    kval = kval.values.reshape([num_krad, num_kang])
    
    orbital_2s = 0
    momentum = krad[:,0]

    num_krad_half = int(num_krad/2)
    num_kang_half = int(num_kang/2)
    dk = np.zeros((num_krad), dtype = 'float')
    for i in range(0,num_krad):
        if(i < num_krad):
            dk[i] = momentum[i] - momentum[i-1]   
        if(i == 0) :
            dk[i] = momentum[i] - 0.0



    def Legendre0():
        first = np.zeros((num_kang), dtype = 'float')

        for i in range(orbital_2s, num_krad_half):
            for j in range(0, num_kang):
                first[j] += kval[i , j] * dk[i] * momentum[i] * momentum[i]
        all = 0
        for j in range(0, num_kang):
            all +=   first[j]  * weight[j] * eval_legendre(0, cos_ang[j])
        return  all / 2
    def Legendre1():
        first = np.zeros((num_kang), dtype = 'float')

        for i in range(orbital_2s, num_krad_half):
            for j in range(0, num_kang):
                first[j] += kval[i , j] * dk[i] * momentum[i] * momentum[i]
        all = 0

        for j in range(0, num_kang):
            all +=   first[j]  * weight[j] * eval_legendre(1, cos_ang[j])

        return  all * 3/2

    def Legendre2():
        first = np.zeros((num_kang), dtype = 'float')

        for i in range(orbital_2s, num_krad_half):
            for j in range(0, num_kang):
                first[j] += kval[i , j] * dk[i] * momentum[i] * momentum[i]
        all = 0

        for j in range(0, num_kang):
            all +=   first[j]  * weight[j] * eval_legendre(2, cos_ang[j])

        return  all * 5/2

    def Legendre3():
        first = np.zeros((num_kang), dtype = 'float')

        for i in range(orbital_2s, num_krad_half):
            for j in range(0, num_kang):
                first[j] += kval[i , j] * dk[i] * momentum[i] * momentum[i]
        all = 0

        for j in range(0, num_kang):
            all +=   first[j]  * weight[j] * eval_legendre(3, cos_ang[j])

        return all * 7/2

    def Legendre4():
        first = np.zeros((num_kang), dtype = 'float')

        for i in range(orbital_2s, num_krad_half):
            for j in range(0, num_kang):
                first[j] += kval[i , j] * dk[i] * momentum[i] * momentum[i]
        all = 0

        for j in range(0, num_kang):
            all +=   first[j]  * weight[j] * eval_legendre(4, cos_ang[j])

        return all * 9/2

    beta0 = Legendre0()
    beta1 = Legendre1()/beta0
    beta2 = Legendre2()/beta0
    beta3 = Legendre3()/beta0
    beta4 = Legendre4()/beta0
    
    beta = pd.DataFrame([[beta1, beta2, beta3, beta4]], columns = ['beta1','beta2', 'beta3', 'beta4'])
    return   beta
      

In [7]:
betas = []
df = pd.DataFrame(index = [], columns = ['beta1','beta2', 'beta3', 'beta4'])

for element in name:
    fname = '/Ne.1.0.8.SPECTRUM/Ne.1.0.8_00000030.ang_spec'
    fname = element + fname 
    row = Beta(fname)
    df = df.append(row)
df['asymmetry']=asym
df.index = ['cep0°','cep45°','cep90°','cep135°','cep180°','cep225°','cep270°','cep315°']

print(df)

df.to_csv('beta_parameters_tdcasscf.txt', mode='w')

            beta1     beta2     beta3     beta4  asymmetry
cep0°   -0.065474 -0.548608  0.371498  0.239384  -0.079691
cep45°   0.298577 -0.548606  0.283820  0.239383   0.113665
cep90°   0.487729 -0.548604  0.029881  0.239381   0.240440
cep135°  0.391180 -0.548604 -0.241565  0.239379   0.226371
cep180°  0.065487 -0.548605 -0.371508  0.239378   0.079699
cep225° -0.298563 -0.548607 -0.283830  0.239378  -0.113657
cep270° -0.487715 -0.548609 -0.029891  0.239380  -0.240431
cep315° -0.391166 -0.548609  0.241554  0.239383  -0.226362
